# Tutorial by Marc LABADIE for software paper rebuild echap loop with adel protocol after conversion in python3

## Create a wheat architectural model with geometry at a given development stage 

In [1]:
from alinea.echap.architectural_reconstructions import EchapReconstructions
import openalea.plantgl.all as pgl
from oawidgets.plantgl import PlantGL

In [2]:
%gui qt

### Initiate the MTG  Echap reconstruction

In [3]:
echap = EchapReconstructions()

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to 

### Get reconstruction according to variety, nb plant and age of the canopy

In [5]:
wheat = echap.get_reconstruction(name="Mercia", nplants=2,seed=0)
g= wheat.setup_canopy(age=2000)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\scipy\interpolate\interpolate.py:446: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = array(y, copy=self.copy)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of li

###  Plot the 3D plant

In [6]:
scene= wheat.plot(g)

In [7]:
PlantGL(scene,group_by_color=False)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera_animation=[], camer…

## Grow canopy

In [13]:
canopy_age= 100 # age of canopy at start
grow_cycles =20 # nb of growth cycle
grow_time = 100 #degree day, time interval between each cycle

wheat = echap.get_reconstruction(name="Tremie12", nplants=2,seed=0) # init a wheat mtg

for cycle in range(grow_cycles):
    canopy_age = canopy_age
    # update canopy
    new_g = wheat.setup_canopy(age=canopy_age)
    scene= wheat.plot(new_g)
    #PlantGL(scene,group_by_color=False) Error:max() arg is an empty sequence
    canopy_age = canopy_age + grow_time

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\scipy\interpolate\interpolate.py:446: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y = array(y, copy=self.copy)
C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of li

# pesticide Interception

In [46]:
import pandas

from alinea.echap.interfaces import pesticide_interception
from alinea.echap.interception_leaf import pesticide_applications, InterceptModel
from alinea.echap.weather_data import get_weather

from alinea.astk.TimeControl import time_filter , rain_filter, thermal_time_filter, DegreeDayModel

### Set weather data and init weather variable

* get wheater data

In [42]:
weather= get_weather(variety="Mercia")
weather.data

,date,h,temperature_air,relative_humidity,rain,wind_speed,global_radiation,PPFD,vapor_pressure
date_utc,,,,,,,,,
2010-09-01 00:00:00+00:00,2010-09-01 00:00:00,00:00,10.1,87.0,0.0,1.666667,0.0,0.0,1.075508
2010-09-01 01:00:00+00:00,2010-09-01 01:00:00,01:00,9.4,90.5,0.0,0.833333,0.0,0.0,1.067407
2010-09-01 02:00:00+00:00,2010-09-01 02:00:00,02:00,8.9,91.5,0.0,1.111111,0.0,0.0,1.043400
2010-09-01 03:00:00+00:00,2010-09-01 03:00:00,03:00,9.0,92.0,0.0,1.666667,0.0,0.0,1.056216
2010-09-01 04:00:00+00:00,2010-09-01 04:00:00,04:00,8.6,93.0,0.0,1.388889,0.0,0.0,1.039185
...,...,...,...,...,...,...,...,...,...
2011-08-31 19:00:00+00:00,2011-08-31 19:00:00,19:00,17.8,59.0,0.0,1.388889,0.0,0.0,1.202524
2011-08-31 20:00:00+00:00,2011-08-31 20:00:00,20:00,16.8,68.0,0.0,1.944444,0.0,0.0,1.301048
2011-08-31 21:00:00+00:00,2011-08-31 21:00:00,21:00,16.0,72.0,0.2,2.222222,0.0,0.0,1.309166


* init weather parameter for pesticide model

In [47]:
periods = 5 # 5000 pour le cycle complet
seq = pandas.date_range(start = "2010-11-02", periods=periods, freq='H')
TTmodel = DegreeDayModel(Tbase = 150)

every_rain = rain_filter(seq, weather)
every_h = time_filter(seq, delay = 6)
every_dd = thermal_time_filter(seq, weather, TTmodel, delay = 15)

C:\Users\mlabadie\miniconda3\envs\openalea3\lib\site-packages\pandas\core\indexing.py:1124: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  return self._get_label(key, axis=axis)


### set application and interception model

* Set application data

In [48]:
applications = """date,dose, product_name
2010-11-02 00:00:00, 0, bug
2010-11-02 1:00:00, 1, Opus
2010-11-02 2:00:00, 0, bug
2010-04-29 10:00:00, 1, Opus
2010-04-29 11:00:00, 0, bug
2010-05-22 10:00:00, 1, Opus
2010-05-22 11:00:00, 0, bug
"""
pesticide_calendar=pesticide_applications(applications)

ProductsDB={'Opus': {'Epoxiconazole': 125}, 'Banko 500': {'Chlorothalonil': 500}}

interception = InterceptModel(ProductsDB)

In [49]:
for cycle in range(grow_cycles):
    canopy_age = canopy_age
    # update canopy
    new_g = wheat.setup_canopy(age=canopy_age)
    pesticide_interception(new_g,interception_model=interception,application_data=pesticide_calendar,label='LeafElement')
    canopy_age = canopy_age + grow_time


IndexError: list index out of range